In [1]:
import os

cwd = os.getcwd()

In [2]:
import pandas as pd


df = pd.read_csv(cwd + '/data/processed/posts_train_test_clean.csv')

In [3]:
df.head()

,Unnamed: 0,Post Link,Title,Body,Tags,CreationDate,Answer Date,AcceptedAnswerId,id,score,complete_text,normalized_text
0,69943,16835963,Sidekiq not processing queue,"<p>What possible reasons can <a href=""https://...",<ruby-on-rails-3><redis><sidekiq>,2013-05-30 12:19:00,2013-05-30 12:53:32,16836794,16836794,71,sidekiq not processing queue what possible rea...,sidekiq process queue possibl reason sidekiq p...
1,79655,31667160,Running docker container : iptables: No chain/...,<p>I'm trying to run a container but I get the...,<docker><port><iptables>,2015-07-28 04:03:53,2015-07-28 04:14:04,31667244,31667244,65,running docker container iptables no chain tar...,run docker contain iptabl chain target match n...
2,97858,3299648,Python Compilation/Interpretation Process,<p>I'm trying to understand the python compile...,<python><compiler-construction><interpreter><p...,2010-07-21 13:21:47,2010-07-21 13:28:54,3299724,3299724,53,python compilation interpretation process i m ...,python compil interpret process tri understand...
3,14078,3664272,Is std::vector so much slower than plain arrays?,<p>I've always thought it's the general wisdom...,<c++><arrays><performance><stl><vector>,2010-09-08 02:38:41,2010-09-08 02:57:40,3664349,3664349,239,is std vector so much slower than plain arrays...,std vector much slower plain array alway thoug...
4,46299,13342123,How to get relative path of a file in visual s...,<p>I am trying to get the path of an image fil...,<c#><visual-studio>,2012-11-12 10:33:32,2013-10-11 10:57:06,19316470,19316470,100,how to get relative path of a file in visual s...,get rel path file visual studio tri get path i...


## Criando os targets

In [4]:
def split_tags(tags):
    tags = tags[1:-1]
    tags = tags.split("><")
    
    return tags

In [5]:
df['targets'] = df.Tags.apply(lambda x: split_tags(x))

In [6]:
X, y = df['complete_text'], df['targets'].apply(lambda x: x[0])

In [7]:
y

0        ruby-on-rails-3
1                 docker
2                 python
3                    c++
4                     c#
              ...       
73917         javascript
73918     android-studio
73919               html
73920         javascript
73921               http
Name: targets, Length: 73922, dtype: object

In [8]:
all_targets = df.targets.apply(lambda x: x[0]).tolist()

In [9]:
from itertools import chain


targets_set = list()

# for targets in all_targets:
#     for target in targets:
#         targets_set.append(target)
        
for targets in all_targets:
    targets_set.append(targets)

In [10]:
targets_set = set(targets_set)

In [11]:
len(targets_set)

1460

In [12]:
targets_set

{'sharepoint',
 'swagger-ui',
 'telegram',
 'android-activity',
 'dropbox',
 'rsync',
 'dataframe',
 'ajp',
 'google-maps-android-api-2',
 'data-structures',
 'gnu-screen',
 'java-native-interface',
 'linq-to-sql',
 'jenkins',
 'unity3d',
 'csh',
 'tinymce',
 'vim',
 'compiler-theory',
 'xcode11',
 'internet-explorer',
 '.net-2.0',
 'julia',
 'google-chrome',
 'code-signing',
 'math',
 'angular',
 'tagging',
 'iphone',
 'checksum',
 'readonly',
 'sorting',
 'homebrew',
 'entity-framework-core',
 '.net',
 'webservice-client',
 'initialization',
 'branch',
 'methods',
 'operating-system',
 'firefox',
 'reverse-debugging',
 'linq',
 'django-models',
 'jstl',
 'uiswitch',
 'laravel-5.2',
 'text-files',
 'code-metrics',
 'windows',
 'ssms',
 'gradle',
 'lets-encrypt',
 'certificate',
 'database-design',
 'collision-detection',
 'rpc',
 'continuous-integration',
 'eclipse',
 'datepicker',
 'haproxy',
 'blogs',
 'razor',
 'cloud',
 'offline',
 'windows-7-x64',
 'easymock',
 'media',
 'google-

In [13]:
words_post = df.normalized_text.apply(lambda x: str(x))

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vectorizer.fit(words_post)
vector = vectorizer.transform(words_post)

In [14]:
import gc

gc.collect()

0

In [15]:
from sklearn.model_selection import train_test_split
    
X_train, X_test, y_train, y_test = train_test_split(vector, y, test_size=0.2, random_state=42)

In [16]:
from sklearn.ensemble import RandomForestClassifier
# rf_model = RandomForestClassifier(max_features="auto", n_jobs=1,random_state=44)
rf_model = RandomForestClassifier(max_features="auto", n_estimators=100,max_depth=48, n_jobs=1,random_state=44)
rf_model.fit(X_train, y_train)

RandomForestClassifier(max_depth=48, n_jobs=1, random_state=44)

In [17]:
y_pred = rf_model.predict(X_test)

In [18]:
gc.collect()

48

In [19]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


C:\Users\Vitor\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Vitor\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                             precision    recall  f1-score   support

                  .htaccess       0.00      0.00      0.00         1
                       .net       1.00      0.01      0.02       127
                   .net-3.5       0.00      0.00      0.00         1
                   .net-4.0       0.00      0.00      0.00         2
           active-directory       0.00      0.00      0.00         2
               afnetworking       0.00      0.00      0.00         1
                       agda       0.00      0.00      0.00         1
                    airflow       0.00      0.00      0.00         4
                       ajax       0.00      0.00      0.00         4
                        ajp       0.00      0.00      0.00         1
                  algorithm       0.00      0.00      0.00        51
            amazon-dynamodb       0.00      0.00      0.00         2
                 amazon-ec2       0.00      0.00      0.00         3
                  amazon-s3      

C:\Users\Vitor\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
# rf_model.base_estimator_
# rf_model.estimators_
len(rf_model.estimators_)

100

In [21]:
# import pickle
# s = pickle.dumps(rf_model)
# rf_model_2 = pickle.loads(s)

In [24]:
from joblib import dump, load
dump(rf_model, 'models/rf_model.joblib') 

['rf_model.joblib']

In [23]:
from sklearn.model_selection import cross_val_score
import numpy as np

np.mean(cross_val_score(rf_model, X_train, y_train, cv=5))

C:\Users\Vitor\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


0.5152273794801672